# Sistema Automatizado de Respostas sobre Gerenciamento de Projetos


- **LinkedIn**: [Rodrigo Viannini](https://www.linkedin.com/in/rodrigo-viannini-datascientist/)


Imagine um escritório de projetos em uma empresa que precisa organizar e responder a perguntas sobre gerenciamento de projetos. Para isso, o time de desenvolvimento criou um sistema que utiliza agentes de IA para planejar, redigir e editar respostas baseadas em um arquivo de texto. Esse sistema é automatizado para lidar com o formato dos dados e buscar informações pertinentes de forma eficiente.

No notebook Jupyter, o processo começa com a instalação de pacotes necessários e a configuração das variáveis de ambiente, incluindo chaves de API. O arquivo `api_keys.txt` é lido para obter a chave da API do OpenAI, que é usada para configurar o ambiente para o agente de IA.

A função `formatar_arquivo` é responsável por garantir que o arquivo de texto com as respostas esteja no formato adequado, removendo caracteres especiais e normalizando o texto. Após formatar o arquivo, a função `obter_resposta_do_arquivo` é utilizada para procurar respostas para tópicos específicos no arquivo formatado.

Três agentes de IA são configurados para ajudar no processo:

- **Planejador**: Define um plano claro para o conteúdo sobre um tópico específico.
- **Redator**: Cria uma resposta direta e clara com base no plano fornecido.
- **Editor**: Revisa a resposta para garantir que esteja clara e precisa.

Esses agentes trabalham juntos em tarefas específicas:

1. Planejar o conteúdo,
2. Escrever uma resposta baseada no planejamento,
3. Editar a resposta para garantir qualidade.

O código também inclui uma função `executar_equipe_com_topico`, que combina as respostas do arquivo com o resultado da execução dos agentes e exibe a resposta final formatada.

Finalmente, o script exibe a resposta para o tópico "Que empresa pode adotar o Gerenciamento de Projetos?" em um formato markdown, utilizando o Jupyter Notebook para apresentar o resultado de maneira visual.

## Resumo do Código em Tópicos

1. **Configuração Inicial**:
   - Instalação dos pacotes necessários (`crewai`, `crewai_tools`, `langchain_community` e `unidecode`).
   - Configuração das variáveis de ambiente e leitura da chave API do OpenAI.

2. **Funções de Manipulação de Arquivos**:
   - `formatar_arquivo`: Formata o arquivo de texto, remove caracteres especiais e ajusta o formato das linhas.
   - `obter_resposta_do_arquivo`: Busca uma resposta para um tópico específico no arquivo formatado.

3. **Configuração dos Agentes**:
   - **Planejador**: Planeja o conteúdo para um tópico específico.
   - **Redator**: Escreve uma resposta com base no plano.
   - **Editor**: Revisa a resposta para garantir clareza e precisão.

4. **Execução e Resultados**:
   - Função `executar_equipe_com_topico`: Combina a resposta do arquivo com o resultado da equipe de agentes e exibe a resposta final.
   - Exibição do resultado no Jupyter Notebook usando Markdown.

5. **Utilitários**:
   - Funções no `utils.py` para carregar variáveis de ambiente e formatar resultados.


## Instalações necessárias

In [54]:
!pip install crewai;
!pip install crewai_tools;
!pip install langchain_community==0.0.29;

## Importação de bibliotecas necessárias

1. **Importação das Bibliotecas**:
   - `os`: Usado para manipular variáveis de ambiente.
   - `crewai`: Usado para criar agentes e tarefas para o sistema.
   - `IPython.display`: Usado para exibir resultados formatados em Markdown.

In [55]:
import os
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew

## OpenAI

2. **Configuração da API**:
   - Define o modelo da OpenAI a ser utilizado como `gpt-4o-mini`.
   - Lê as chaves de API de um arquivo `api_keys.txt` e define as variáveis de ambiente necessárias.

### Define o modelo da OpenAI a ser utilizado

In [56]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

### Ler chaves de APIs do arquivo texto

In [57]:
# Abre o arquivo 'api_keys.txt' e lê as chaves de API
with open('api_keys.txt', 'r') as file:
    lines = file.readlines()  # Lê todas as linhas do arquivo
    openai_api_key = lines[0].strip()  # Obtém a chave de API do OpenAI da primeira linha e remove espaços
    # serper_api_key = lines[1].strip()  # Descomente se precisar usar a chave do Serper na segunda linha
    print(lines)  # Imprime as linhas para verificação (pode ser removido em produção)


['sk-proj-55k7heMO5uPMUFbKPBLdT3BlbkFJmnELafKh5akW0Acklyyl']


### Define a chave de API do OpenAI no ambiente

In [58]:
os.environ["OPENAI_API_KEY"] = openai_api_key

## Arquivo de conhecimento

In [59]:
# Caminho para o arquivo de texto com as respostas
caminho_arquivo = 'gerenciamento_de_projetos.txt'

#### Nova instalação

In [60]:
!pip install unidecode

#### Utilização de uma lib especifica

In [61]:
from unidecode import unidecode
import re

In [62]:
def formatar_arquivo(caminho_arquivo):
    """
    Formata o arquivo de texto para garantir que cada linha siga o formato
    'tópico: resposta' e remove espaços indesejados e caracteres especiais.

    Parâmetros:
    caminho_arquivo (str): O caminho para o arquivo de texto a ser formatado.

    Retorna:
    novo_caminho_arquivo (str): O caminho para o novo arquivo formatado.
    """
    try:
        # Lê o conteúdo do arquivo original
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            linhas = arquivo.readlines()

        # Lista para armazenar as linhas formatadas
        linhas_formatadas = []

        # Processa cada linha do arquivo
        for linha in linhas:
            # Remove espaços em branco no início e fim da linha
            linha = linha.strip()
            # Remove caracteres indesejados e deixa apenas pontuações e letras
            linha = re.sub(r'[^a-zA-Z0-9.,:;!? ]', '', linha)
            # Remove acentos e caracteres especiais
            linha = unidecode(linha)
            # Converte o texto para minúsculas
            linha = linha.lower()
            
            if ':' in linha:
                # Divide a linha em tópico e resposta
                chave, valor = linha.split(':', 1)
                # Remove espaços desnecessários e padroniza
                chave = chave.strip()
                valor = valor.strip()
                # Padroniza a formatação do texto
                valor = valor.replace('  ', ' ')  # Remove espaços duplos
                valor = valor.replace('.', '. ')  # Garante espaço após ponto final
                # Reconstrói a linha formatada
                linha_formatada = f"{chave}: {valor}"
                linhas_formatadas.append(linha_formatada)
            else:
                # Se a linha não contiver ':', ignora ou registra um aviso
                print(f"Aviso: Linha ignorada por falta de delimitador ':': {linha}")

        # Define um novo caminho para o arquivo formatado
        novo_caminho_arquivo = caminho_arquivo.replace('.txt', '_formatado.txt')

        # Escreve o conteúdo formatado para um novo arquivo
        with open(novo_caminho_arquivo, 'w', encoding='utf-8') as arquivo:
            for linha_formatada in linhas_formatadas:
                arquivo.write(linha_formatada)

        print(f"Formatação do arquivo concluída com sucesso. Novo arquivo: {novo_caminho_arquivo}")

        return novo_caminho_arquivo

    except FileNotFoundError:
        print(f"Erro: O arquivo {caminho_arquivo} não foi encontrado.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

In [63]:
caminho_arquivo = 'gerenciamento_de_projetos.txt'
novo_caminho_arquivo = formatar_arquivo(caminho_arquivo)

Aviso: Linha ignorada por falta de delimitador ':': 1. introduo ao gerenciamento de projetos
Aviso: Linha ignorada por falta de delimitador ':': 1.1 definio e conceitos bsicos
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 1.2 histria e evoluo do gerenciamento de projetos
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 1.3 importncia do gerenciamento de projetos
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 1.4 papis e responsabilidades do gerente de projetos
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitador ':': 
Aviso: Linha ignorada por falta de delimitad

In [64]:
# Certifique-se de que a variável está definida antes de usar
if caminho_arquivo is None:
    raise ValueError("O caminho do arquivo não foi definido. Certifique-se de que 'caminho_arquivo' está atribuído corretamente.")

## Funções Utilitárias

### Função para formatar a resposta em linhas de até 80 caracteres

1. **`formatar_resultado(resposta)`**:
   - Formata uma string de resposta para garantir que cada linha tenha no máximo 80 caracteres.
   - Divide as linhas longas em palavras e reorganiza em linhas de comprimento adequado.

### Função para obter a resposta do arquivo de texto baseado no tópico fornecido

2. **`obter_resposta_do_arquivo(topico, caminho_arquivo)`**:
   - Lê um arquivo de texto que contém tópicos e suas respectivas respostas.
   - Pesquisa a resposta para um tópico específico e limita a saída a X caracteres.
   - Retorna a resposta formatada ou uma mensagem de erro se o tópico não for encontrado.

### Quantidad ede Caracteres do output: 

In [65]:
qtde_caracteres = 500 

In [66]:
def obter_resposta_do_arquivo(topico, caminho_arquivo):
    """
    Obtém uma resposta para um tópico específico a partir de um arquivo de texto.
    
    Parâmetros:
    topico (str): O tópico a ser pesquisado.
    caminho_arquivo (str): O caminho para o arquivo de texto contendo as respostas.
    
    Retorna:
    str: A resposta encontrada ou uma mensagem indicando que o tópico não foi encontrado.
    """
    try:
        # Tenta abrir o arquivo no modo leitura com codificação UTF-8
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            # Lê todas as linhas do arquivo
            linhas = arquivo.readlines()

            # Itera sobre cada linha do arquivo
            for linha in linhas:
                # Remove espaços em branco do início e do fim da linha
                linha = linha.strip()

                # Verifica se a linha contém um separador de chave e valor
                if ':' in linha:
                    # Divide a linha em chave e valor
                    chave, valor = linha.split(':', 1)
                    # Remove espaços em branco da chave e converte para minúsculas
                    chave = chave.strip().lower()
                    # Remove espaços em branco do valor
                    valor = valor.strip()

                    # Verifica se a chave corresponde ao tópico pesquisado
                    if chave == topico.strip().lower():
                        # Limita a resposta a qtde_caracteres caracteres
                        resposta = valor[:qtde_caracteres]
                        # Retorna a resposta encontrada
                        return resposta

        # Se o tópico não foi encontrado, junta todo o conteúdo das linhas em uma única string
        conteudo_completo = ''.join(linhas)
        # Retorna uma mensagem indicando que o tópico não foi encontrado e exibe o conteúdo completo
        return f"{conteudo_completo}\nDesculpe, não encontrei uma resposta para esse tópico."

    except FileNotFoundError:
        # Retorna uma mensagem de erro se o arquivo não for encontrado
        return f"Erro: O arquivo {caminho_arquivo} não foi encontrado."
    except Exception as e:
        # Retorna uma mensagem de erro para qualquer outra exceção
        return f"Ocorreu um erro ao acessar o arquivo: {e}"


## Criação dos agentes com a chave de API

1. **Agentes Criados**:
   - **Planejador**: Responsável por planejar o conteúdo.
   - **Redator**: Escreve o conteúdo baseado no plano.
   - **Editor**: Edita o conteúdo para garantir clareza e consistência.

In [67]:
# Agente: Planejador
planejador = Agent(
    role="Planejador de Conteúdo",  # Define o papel do agente
    goal="Planejar conteúdo claro e objetivo sobre o tópico: {topic}",  # Define o objetivo do agente
    backstory="Você está planejando um conteúdo sobre o tópico: {topic}.",  # Histórico do agente
    allow_delegation=False,  # Não permite delegação de tarefas
    verbose=True,  # Ativa o modo detalhado para saída de logs
    openai_api_key=openai_api_key  # Define a chave de API para o agente
)

# Agente: Redator
redator = Agent(
    role="Redator de Conteúdo",
    goal="Escrever uma resposta concisa e clara sobre o tópico: {topic}",
    backstory="Você está redigindo uma resposta direta e precisa sobre o tópico: {topic}.",
    allow_delegation=False,
    verbose=True,
    openai_api_key=openai_api_key  # Define a chave de API para o agente
)

# Agente: Editor
editor = Agent(
    role="Editor",
    goal="Editar a resposta para garantir clareza, precisão e conformidade com o estilo.",
    backstory="Você revisa a resposta para garantir que esteja bem estruturada e coerente.",
    allow_delegation=False,
    verbose=True,
    openai_api_key=openai_api_key  # Define a chave de API para o agente
)

## Criação das tarefas

2. **Tarefas Definidas**:
   - **Planejar**: Identificar e estruturar pontos principais sobre um tópico.
   - **Escrever**: Criar uma resposta concisa baseada no plano.
   - **Editar**: Revisar a resposta para garantir qualidade e precisão.

In [68]:
# Tarefa: Planejar
planejar = Task(
    description=(
        "1. Identificar os principais pontos sobre o tópico: {topic}.\n"
        "2. Priorizar informações relevantes e concisas.\n"
        "3. Desenvolver um esboço claro e objetivo.\n"
        "4. Garantir que a resposta tenha no máximo qtde_caracteres."
    ),
    expected_output="Um plano de conteúdo claro e conciso com tópicos estruturados em português.",
    agent=planejador,
)


# Tarefa: Escrever
escrever = Task(
    description=(
        "1. Criar uma resposta direta sobre o tópico: {topic}.\n"
        "2. Usar o plano de conteúdo como base.\n"
        "3. Garantir clareza e precisão na resposta.\n"
        "4. Limitar a resposta a qtde_caracteres.\n"
        "5. Escrever o conteúdo em português de forma coerente."
    ),
    expected_output="Uma resposta clara e direta em formato de texto, escrita em português.",
    agent=redator,
)

# Tarefa: Editar
editar = Task(
    description=("Revisar a resposta para clareza, precisão e garantir que esteja em português."),
    expected_output="Uma resposta editada e pronta para uso, em português.",
    agent=editor
)

## Criação da equipe (Crew)

3. **Configuração da Equipe**:
   - A equipe é composta por todos os agentes e suas respectivas tarefas, que serão executadas sequencialmente.


In [69]:
equipe = Crew(
    agents=[planejador, redator, editor],  # Lista de agentes na equipe
    tasks=[planejar, escrever, editar],  # Lista de tarefas a serem executadas
    verbose=2  # Nível de detalhe dos logs
)

2024-08-07 13:41:50,972 - 6456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


## Execução da Equipe

### Função de execução da equipe com base no tópico fornecido

1. **Função `executar_equipe_com_topico`**:
   - Executa a equipe de agentes para um tópico fornecido.
   - Lê a resposta correspondente do arquivo de texto.
   - Executa as tarefas da equipe e obtém o resultado do processo.
   - Combina e formata a resposta do arquivo e o result

In [71]:
def executar_equipe_com_topico(topico, caminho_arquivo):
    """
    Executa a equipe de agentes para um tópico específico e retorna o resultado formatado em tópicos.

    Parâmetros:
    topico (str): O tópico sobre o qual a equipe irá trabalhar.
    caminho_arquivo (str): O caminho para o arquivo de texto contendo as respostas.

    Retorna:
    str: O resultado combinado da execução da equipe e a resposta do arquivo, formatados em tópicos.
    """
    # Obtém a resposta do arquivo de respostas
    resposta_do_arquivo = obter_resposta_do_arquivo(topico, caminho_arquivo)

    # Executa a equipe e obtém o resultado
    resultado = equipe.kickoff(inputs={"topic": topico})

    # Limita o resultado da equipe a qtde_caracteres por tópico
    resultado_limpo = resultado.strip()[:qtde_caracteres]

    # Formata a resposta em tópicos de até qtde_caracteres
    def formatar_texto_em_topicos(texto):
        topicos = texto.split('. ')
        topicos_formatados = []
        for topico in topicos:
            if len(topico) > qtde_caracteres:
                # Divide em partes menores se o tópico exceder qtde_caracteres
                partes = [topico[i:i+qtde_caracteres] for i in range(0, len(topico), qtde_caracteres)]
                topicos_formatados.extend(partes)
            else:
                topicos_formatados.append(topico)
        return '\n'.join(topicos_formatados)

    resposta_formatada = formatar_texto_em_topicos(resposta_do_arquivo)
    resultado_formatado = formatar_texto_em_topicos(resultado_limpo)

    # Combina a resposta do arquivo com o resultado da execução da equipe
    resultado_completo = (
        # f"**Resposta do Arquivo:**\n{resposta_formatada}\n\n"
        f"**Resultado da Equipe:**\n{resultado_formatado}"
    )

    return resultado_completo

### Executar a equipe com um tópico específico

2. **Execução da Equipe**:
   - O tópico "DIGITE SEU TOPICO" é usado como exemplo.
   - A equipe executa as tarefas definidas para gerar uma resposta estruturada.

In [79]:
topico1 = "O que define um projeto e como ele se diferencia de operações contínuas?"
resultado1 = executar_equipe_com_topico(f'escreva em ate {qtde_caracteres} caracteres: {topico1}', novo_caminho_arquivo)

 [DEBUG]: == Working Agent: Planejador de Conteúdo
 [INFO]: == Starting Task: 1. Identificar os principais pontos sobre o tópico: escreva em ate 500 caracteres: O que define um projeto e como ele se diferencia de operações contínuas?.
2. Priorizar informações relevantes e concisas.
3. Desenvolver um esboço claro e objetivo.
4. Garantir que a resposta tenha no máximo qtde_caracteres.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: Um projeto é definido como um esforço temporário para criar um produto, serviço ou resultado único. Ele possui um objetivo específico, prazo determinado, recursos definidos e uma equipe designada. Por outro lado, operações contínuas são atividades repetitivas e rotineiras, que mantêm a organização funcionando no dia a dia. A principal diferença entre projeto e operações contínuas está na natureza temporária e única do projeto, enquanto as operações são contínuas e repetitivas. É essencial compreender essa distinção para

In [80]:
topico2 = """Quais são as principais fases do ciclo de vida de um projeto e qual a importância de cada uma?
Descreva as fases do ciclo de vida do projeto e a importância de cada fase no gerenciamento eficaz do projeto."""
resultado2 = executar_equipe_com_topico(f'escreva em ate {qtde_caracteres} caracteres: {topico2}', novo_caminho_arquivo)

## Exibição de Resultados

3. **Exibição de Resultados**:
   - O resultado combinado é exibido usando a biblioteca `IPython.display` para formatação em Markdown.

In [81]:
# Exibir os resultados da execução
from IPython.display import display, Markdown
display(Markdown(resultado1))

**Resultado da Equipe:**
Um projeto é um esforço temporário com objetivo específico, prazo, recursos definidos e equipe designada, visando criar um produto, serviço ou resultado único
Já operações contínuas são atividades rotineiras e repetitivas que mantêm a organização funcionando
A diferença principal está na natureza temporária e única do projeto em contraste com a continuidade das operações
É crucial entender essa distinção para gerenciar adequadamente cada tipo de atividade.

In [78]:
# # Exibir os resultados da execução
# from IPython.display import display, Markdown
# display(Markdown(resultado2))

**Resultado da Equipe:**
As principais fases do ciclo de vida de um projeto são: iniciação, planejamento, execução, monitoramento e controle, e encerramento
A iniciação define o escopo e objetivos do projeto
O planejamento estabelece atividades, recursos e cronograma
A execução é onde o projeto é implementado
O monitoramento e controle permitem ajustes e acompanhamento do progresso
O encerramento formaliza a conclusão e avalia o desempenho, garantindo o sucesso do projeto.